In [2]:
from preporcess import CleanerAnny
from __init__ import confUser

In [6]:
confUser['DATADIR'] = 'data/tempdir/'
cleannerAnny = CleanerAnny(confUser, ['金融资产'], v=False)
dfYall = cleannerAnny.startCleaning(intresedCode= [107, 130, 170])

data/tempdir/rawdata
11596
(69481, 6)


/Users/xl-macbook/anaconda/lib/python2.7/site-packages/pandas/core/common.py:452: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
  mask = arr == x
/Users/xl-macbook/anaconda/lib/python2.7/site-packages/pandas/core/common.py:464: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
  mask |= arr == x


KeyError: '\xe9\x87\x91\xe8\x9e\x8d\xe8\xb5\x84\xe4\xba\xa7\xe4\xbb\xa3\xe7\xa0\x81'